In [8]:
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import os
import difflib


In [9]:
audl = pd.concat([pd.read_csv(dfteam, index_col=0) for dfteam in glob.glob('../output/*.csv')])

In [3]:
webscraped_players = pd.read_csv('01_webscraped_audlplayers2018.csv')

somelist=[]
for i in webscraped_players['team-href'].values:
    result = requests.get("{}/players".format(i))
    soup = BeautifulSoup(result.text,'lxml')
    for j in (soup.find_all('div', {'class':"views-field views-field-player-field-player-display-name"})):
        somelist.append([os.path.basename(i),j.text])

In [19]:
rosters2018 = pd.DataFrame(somelist,columns=['team','PlayerName'])
rosters2018['PlayerName'] = rosters2018.PlayerName.str.rstrip(' ').str.lstrip(' ')
rosters2018['Tournament'] = 'AUDL 2018'

tpdict={}
for team in rosters2018.team.unique():
    mtch = difflib.get_close_matches(team,[ teamname.split()[-1] for teamname in audl.Teamname.unique()],cutoff=.5 )[0]
    tmnm = difflib.get_close_matches(mtch,audl.Teamname.unique(),cutoff=.4)[0]
    tpdict[team]=tmnm
    
rosters2018['Teamname'] = [tpdict[i] for i in rosters2018.team]


In [23]:
rosters2018.to_csv('02_roster2018.csv')
